In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

In [4]:
def computeH(points_im1, points_im2):
    matrixA = []
    for i in range(len(points_im1)):
        xs, ys = points_im1[i][0], points_im1[i][1]
        xd, yd = points_im2[i][0], points_im1[i][1]
        two_nineMatrix = np.array([[xs, ys, 1,0,0,0, -xd*xs, -xd*ys, -xd],
                                   [0,0,0,xs,ys,1,-yd*xs,-yd*ys,-yd]])
        matrixA.append(two_nineMatrix[0])
        matrixA.append(two_nineMatrix[1])
    return matrixA

In [5]:
def interpolation(image, x,y):
    leftTopX, leftTopY, leftTopVal = int(x), int(y), image[int(x)][int(y)]
    rightTopX, rightTopY, rightTopVal = int(x) +1 , int(y), image[int(x)+1][int(y)]
    leftBottomX, leftBottomY, leftBottomVal = int(x), int(y)+1, image[int(x)][int(y)+1]
    rightBottomX, rightBottomY, rightBottomVal = int(x)+1, int(y)+1, image[int(x)+1][int(y)+1]
    a=x%1
    b= int(y)+1 - y
    return leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b

In [6]:
def pixelIntensity(rgb):
     return 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]

In [7]:
def blend(pixel1,pixel2):
    if pixelIntensity(pixel1) > pixelIntensity(pixel2):
        return pixel1
    else:
        return pixel2

In [8]:
def warp(image,homography):
    t_inverse = np.linalg.inv(homography)
    imHeight = len(image)
    imWidth = len(image[0])
    boundries = np.matmul(homography, np.array([[0,0,imWidth,imWidth],
                                                [0,imHeight,0,imHeight],
                                                [1,1,1,1]]))
    boundriesDenormalized = np.matmul(boundries, np.diag(1/boundries[-1]))
    #print(boundriesDenormalized, np.shape(boundriesDenormalized))
    inversed = np.matmul(t_inverse, boundriesDenormalized)
    inversedNormalised = np.matmul(inversed, np.diag(1/inversed[-1]))
    #print(inversedNormalised)
    newPositions = []
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    minXIndex = np.where(boundriesDenormalized[0] == np.amin(boundriesDenormalized[0]))
    #print('minx',minXIndex, boundriesDenormalized[0][minXIndex], boundriesDenormalized[1][minXIndex] )
    coordinates1 = np.matmul(t_inverse, np.array([[-539],[-136],[1]]))
    #print("coordinates1",coordinates1 * (1/coordinates1[2][0]) )
    yAxisRange = (int(np.amin(boundriesDenormalized[1])),int(np.amax(boundriesDenormalized[1])))
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    #print(xAxisRange,yAxisRange)
    width = xAxisRange[1]-xAxisRange[0]
    height = yAxisRange[1]-yAxisRange[0]
    shiftX = -xAxisRange[0] if xAxisRange[0] < 0 else 0
    shiftY = -yAxisRange[0] if yAxisRange[0] < 0 else 0
    wrapped = np.zeros((yAxisRange[1]+shiftY+1,xAxisRange[1]+shiftX+1,3))
    #print(height, width)
    #print('traverseIndex',height-1,width-1)
    for h in range(len(wrapped)-1):
        for w in range(len(wrapped[0])-1):
            x = w - shiftX
            y = h - shiftY
            coordinates = np.matmul(t_inverse, np.array([[x],[y],[1]]))
            coordinates = (coordinates* (1/coordinates[2][0]))[0:2]
            if coordinates[0] > 0 and coordinates[1] > 0:
                if int(coordinates[1]) < len(image)-1 and int(coordinates[0]) < len(image[0])-1:
                    bilinearInterpolation  = interpolation(image,coordinates[1][0],coordinates[0][0])
                    wrapped[h][w] = bilinearInterpolation
    return np.asarray(wrapped), (shiftX,shiftY), (xAxisRange,yAxisRange)

In [9]:
def firstElements(lst): 
    return [x[0] for x in lst]

In [10]:
def secondElements(lst):
    return [x[1] for x in lst]

In [23]:
def mergeImages(imageList, shiftList, x_yRange):
    leftBound = min(firstElements(firstElements(x_yRange)))
    rightBound = max(secondElements(firstElements(x_yRange)))
    topBound = min(firstElements(secondElements(x_yRange)))
    bottomBound = max(secondElements(secondElements(x_yRange)))
    xAxisRange= (leftBound, rightBound)
    yAxisRange = (topBound, bottomBound)
    print(leftBound,rightBound, topBound, bottomBound)
    merged = np.zeros(( bottomBound - topBound, rightBound - leftBound, 3))
    print(np.shape(merged))
    shiftX = -leftBound if leftBound < 0 else 0
    shiftY = -topBound if topBound < 0 else 0
    for imageIndex,image in enumerate(imageList):
        XtoBeShifted = abs(leftBound) - shiftList[imageIndex][0]
        YtoBeShifted = abs(topBound) - shiftList[imageIndex][1]
        print(imageIndex,XtoBeShifted,YtoBeShifted)
        for row in range(len(image)-1):
            for column in range(len(image[row])-1):
                x=XtoBeShifted + column
                y=YtoBeShifted + row
                #if row == 0 or column == 0 or row == len(image)-2 or column == len(image[row])-2:
                merged[y][x] = blend(image[row][column],merged[y][x])
                #else:
                #    merged[y][x] = image[row][column]
    return merged,(shiftX,shiftY),(xAxisRange,yAxisRange)

In [12]:
def fwarp(image,homography):
    wrapped = np.zeros(np.shape(image))
    newPositions = []
    for i in range(len(image)):
        for j in range(len(image[i])):
            coordinates = np.matmul(homography, np.array([[i],[j],[1]]))
            newPositions.append(coordinates* (1/coordinates[2][0]))
    return np.asarray(newPositions).reshape(np.shape(np.asarray(image)))[:,:,0:2]

In [13]:
def construct(image,wrap):
    res = np.round(wrap)
    x_min = np.amin(res[:,:,0])
    y_min = np.amin(res[:,:,1])
    res[:,:,0] = res[:,:,0] + np.abs(x_min)
    res[:,:,1] = res[:,:,1] + np.abs(y_min)
    x_max = np.amax(res[:,:,0]).astype(int)
    y_max = np.amax(res[:,:,1]).astype(int)
    res = res.astype(int)
    print(x_max,y_max)
    scale_X = (len(image) / x_max) 
    scale_Y = (len(image[0]) / y_max)
    projected = np.zeros(np.shape(image))
    for i in range(len(image)):
        for j in range(len(image[i])):
            c = res[i][j]
            #print(int(c[0] * scale_X), int(c[1] * scale_Y) )
            projected[int(c[0] * scale_X)-1][int(c[1] * scale_Y)-1] = image[i][j]
    return projected

In [14]:
def homography(fileName):
    with open(fileName, 'rb') as f:
        p23_3 = np.load(f)
        p23_2 = np.load(f)
    
    A = computeH(p23_3,p23_2)
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    return vh[-1].reshape(3,3)

In [15]:
def printImage(imageMatrix,outFileName="output.jpg"):
    imgResult = Image.fromarray(imageMatrix.astype(np.uint8), 'RGB')
    imgResult.save(outFileName)

## Paris

In [163]:
imageParis1 = Image.open("paris/paris_a.jpg")
imageParis2 = Image.open("paris/paris_b.jpg")
imageParis3 = Image.open("paris/paris_c.jpg")
img1Paris = np.asarray(imageParis1)
img2Paris= np.asarray(imageParis2)
img3Paris = np.asarray(imageParis3)

In [168]:
homography_12 = homography('paris1-2_15points.npy')
homography_32 = homography('paris2-3_10points.npy')

In [173]:
warpRight,offsetRight,coordinateRangeRight = warp(img3Paris,homography_32)
warpLeft,offsetLeft,coordinateRangeLeft = warp(img1Paris,homography_12)

In [174]:
mergedAll, shifts, = mergeImages([warpRight,warpLeft,img2Paris],[offsetRight,offsetLeft,(0,0)],[coordinateRangeRight,coordinateRangeLeft,((0,len(img2Paris[0])),(0,len(img2Paris)))])
printImage(mergedAll,'paris/parisOutput.jpg')

-627 995 -237 599
(836, 1622, 3)
0 627 174
1 0 0
2 627 237


## NORTH Campus

In [27]:
image1 = Image.open("north_campus/left_2.jpg")
image2 = Image.open("north_campus/left_1.jpg")
image3 = Image.open("north_campus/middle.jpg")
image4 = Image.open("north_campus/right_1.jpg")
image5 = Image.open("north_campus/right_2.jpg")
img1 = np.asarray(image1)
img2 = np.asarray(image2)
img3 = np.asarray(image3)
img4 = np.asarray(image4)
img5 = np.asarray(image5)

In [28]:
homography_12 = homography('campusleft2_left1_10points.npy')
homography_23 = homography('campusleft1_middle_5points.npy')
homography_43 = homography('campusright1_middle_5points.npy')
homography_54 = homography('campusright2_right1_10points.npy')

### Left1 and Right1 to Middle

In [29]:
warpRight,offsetRight,coordinateRangeRight = warp(img4,homography_43)
warpLeft,offsetLeft,coordinateRangeLeft = warp(img2,homography_23)

In [30]:
mergedAll,shifts, ranges = mergeImages([warpRight,warpLeft,img3],[offsetRight,offsetLeft,(0,0)],[coordinateRangeRight,coordinateRangeLeft,((0,len(img3[0])),(0,len(img3)))])
printImage(mergedAll,'north_campus/middle_out/mosaic_1.jpg')

-975 2057 -84 866
(950, 3032, 3)
0 975 18
1 0 0
2 975 84


In [200]:
printImage(warpLeft,'north_campus/warped_left.jpg')
mergedAll,shifts,ranges = mergeImages([warpLeft,img3],[offsetLeft,(0,0)],[coordinateRangeLeft,((0,len(img3[0])),(0,len(img3)))])
printImage(mergedAll,'north_campus/middle_out/mosaic_left.jpg')

-975 1280 -84 866
(950, 2255, 3)
0 0 0
1 975 84


In [31]:
wr_left2,offset4,coordinateRange4 = warp(img1,np.matmul(homography_23,homography_12))
wr_right2,offset5,coordinateRange5 = warp(img5,np.matmul(homography_43,homography_54))

In [32]:
mergedImages = mergeImages([wr_right2,wr_left2,mergedAll],[offset5,offset4,shifts],[coordinateRange4,coordinateRange5,ranges])
imgResult = Image.fromarray(mergedImages.astype(np.uint8), 'RGB')
imgResult.save("north_campus/mergedImages.jpg")

-8039 2842 -1112 1595
(2707, 10881, 3)
0 8039 953
1 0 0
2 7064 1028


AttributeError: 'tuple' object has no attribute 'astype'

In [135]:
image_mosaic = Image.open("north_campus/north_campus_mosaic1.jpg")
img_msc = np.asarray(image_mosaic)
mergedImages = mergeImages([wr_right2,wr_left2,img_msc],[offset5,offset4,(0,0)],[coordinateRange4,coordinateRange5,((0,len(img_msc[0])),(0,len(img_msc)))])
imgResult = Image.fromarray(mergedImages.astype(np.uint8), 'RGB')
imgResult.save("north_campus/mergedImages3.jpg")

-8039 3032 -1112 1595
(2707, 11071, 3)
0 8039 953
1 0 0
2 8039 1112


## CMPE Building

In [16]:
image1Cmpe = Image.open("cmpe_building/left_2.jpg")
image2Cmpe = Image.open("cmpe_building/left_1.jpg")
image3Cmpe= Image.open("cmpe_building/middle.jpg")
image4Cmpe= Image.open("cmpe_building/right_1.jpg")
image5Cmpe = Image.open("cmpe_building/right_2.jpg")
img1Cmpe = np.asarray(image1Cmpe)
img2Cmpe = np.asarray(image2Cmpe)
img3Cmpe = np.asarray(image3Cmpe)
img4Cmpe = np.asarray(image4Cmpe)
img5Cmpe = np.asarray(image5Cmpe)

In [17]:
homography_12 = homography('cmpe_building/cmpeleft2_left1_10points.npy')
homography_23 = homography('cmpe_building/cmpeleft1_middle_30points.npy')
homography_43 = homography('cmpe_building/cmperight1_middle_30points.npy')
homography_54 = homography('cmpe_building/cmperight2_right1_10points.npy')

### Left1 and Right1 to Middle

In [18]:
warpRight,offsetRight,coordinateRangeRight = warp(img4Cmpe,homography_43)
warpLeft,offsetLeft,coordinateRangeLeft = warp(img2Cmpe,homography_23)

In [25]:
mergedAll,shifts, ranges = mergeImages([warpRight,warpLeft,img3Cmpe],[offsetRight,offsetLeft,(0,0)],[coordinateRangeRight,coordinateRangeLeft,((0,len(img3Cmpe[0])),(0,len(img3Cmpe)))])
printImage(mergedAll,'cmpe_building/middle_out/mosaic_1.jpg')

-231 1617 -48 1035
(1083, 1848, 3)
0 231 13
1 0 0
2 231 48


### Left2 and Right2 to Mosaic

In [20]:
warpRight2,offsetRight2,coordinateRangeRight2 = warp(img5Cmpe,np.matmul(homography_43,homography_54))
print(np.shape(warpRight2))
warpLeft2,offsetLeft2,coordinateRangeLeft2 = warp(img1Cmpe,np.matmul(homography_23,homography_12))

(1498, 2320, 3)


In [26]:
mergedTotal,shiftsT, rangesT = mergeImages([warpRight2,warpLeft2,mergedAll],[offsetRight2,offsetLeft2,shifts],[coordinateRangeRight2,coordinateRangeLeft2,ranges])
printImage(mergedTotal,'cmpe_building/middle_out/totalwithMosaic.jpg')

-519 2319 -123 1374
(1497, 2838, 3)
0 519 0
1 0 25
2 288 75
